**My Task** is to Create a jupyter notebook which will **analyzes the Canadian Cheese directory data set** cross referenced with temperature in the provinces.

Furthermore, I have to answer:
1. Whether there are any inferences I can make about the relationship between the weather in a province and the cheese produced or not.
2. I have to Create two different data visualizations. These can be charts or whatever I would like.
3. For Bonus points, I can provide demonstration of any sort of data pipeline or cleaning skills.

# **Notebook Header & Introduction**

In [1]:
# Canadian Sheep Federation Internship Assessment – Data Analyst Role
# Data Analysis: Canadian Cheese Directory vs. Provincial Weather
# Author: B.M Jahidul Haque
# Date: 7th June 2025
# Tools: Python, Pandas, Matplotlib, Seaborn, Plotly
#
# Notebook Goals:
# - Investigate the relationship between temperature and cheese production across Canadian provinces.
# - Apply professional data analysis and visualization practices.
# - Showcase skills in data cleaning, exploration, visualization, and inference.
#
# Data Sources:
# - https://www.kaggle.com/datasets/noahjanes/canadian-cheese-directory
# - https://www.kaggle.com/datasets/hemil26/canada-weather


# **Install & Import Essentials**

In [2]:
# Install visualization packages
!pip install plotly --quiet

# Core libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Configs
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# General display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


# **Upload and Load Data**

In [3]:
from google.colab import files
uploaded = files.upload()  # This allows multiple files as I have to upload multiple files

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Load Cheese Data
try:
    cheese_df = pd.read_csv("cheese_data.csv")
    print(f"Cheese dataset loaded: {cheese_df.shape[0]} rows, {cheese_df.shape[1]} columns.")
except Exception as e:
    print("Error loading cheese dataset:", e)

# Load Weather Data
try:
    weather_df = pd.read_csv("canada_weather.csv")
    print(f"Weather dataset loaded: {weather_df.shape[0]} rows, {weather_df.shape[1]} columns.")
except Exception as e:
    print("Error loading weather dataset:", e)


# **Data Inspection & Cleaning**

In [ ]:
# Cheese Dataset Overview
print("Cheese Dataset Preview:")
display(cheese_df.head())

print("\n Basic Info:")
cheese_df.info()

print("\n Missing Values:")
print(cheese_df.isnull().sum())

print("\n Unique values per column:")
print(cheese_df.nunique())


In [ ]:
# Weather Dataset Overview
print("Weather Dataset Preview:")
display(weather_df.head())

print("\n Basic Info:")
weather_df.info()

print("\n Missing Values:")
print(weather_df.isnull().sum())

print("\n Unique values per column:")
print(weather_df.nunique())


# **Data Cleaning & Preparation**

In [ ]:
# I’ll focus on:
# Renaming inconsistent columns
# Dropping irrelevant data
# Ensuring province names match
# Handling missing values
# Creating a clean, ready-to-analyze dataframe

In [ ]:
print(" Current cheese_df columns:")
print(cheese_df.columns.tolist())


In [ ]:
# Dynamic Renaming
rename_map = {
    'manufacturerprovcode': 'province',
    'moisturepercent': 'moisture_percent',
    'fatlevel': 'fat_level',
    'categorytypeen': 'category_type',
    'milktypeen': 'milk_type',
    'flavouren': 'flavour',
    'rindtypeen': 'rind_type',
    'cheesename': 'cheese_name',
    'characteristicsen': 'characteristics',
    'milktreatmenttypeen': 'milk_treatment',
    'manufacturingtypeen': 'manufacturing_type'
}

valid_renames = {k: v for k, v in rename_map.items() if k in cheese_df.columns}
cheese_df.rename(columns=valid_renames, inplace=True)

# Drop columns with >30% missing
missing_ratio = cheese_df.isnull().mean()
high_missing_cols = missing_ratio[missing_ratio > 0.3].index.tolist()
cheese_df.drop(columns=high_missing_cols, inplace=True, errors='ignore')

# Drop rows with nulls in important fields (if present)
required_cols = ['moisture_percent', 'category_type', 'milk_type']
valid_required_cols = [col for col in required_cols if col in cheese_df.columns]
cheese_df.dropna(subset=valid_required_cols, inplace=True)

# Normalize text values safely
for text_col in ['fat_level', 'province']:
    if text_col in cheese_df.columns:
        if text_col == 'fat_level':
            cheese_df[text_col] = cheese_df[text_col].astype(str).str.strip().str.lower()
        else:
            cheese_df[text_col] = cheese_df[text_col].astype(str).str.strip().str.title()

# Summary
print("Final Cheese Dataset Columns (after cleaning):")
print(cheese_df.columns.tolist())
print(f"Final shape: {cheese_df.shape}")
cheese_df.head()


# **weather dataset cleaning:**

Has temperature in "value (°F)" format, e.g., "10.8 (51.4)"

Has no separate province column, only Community names like "Charlottetown, PE"

In [ ]:
import re

#  Extract Province Code
weather_df['province'] = weather_df['Community'].apply(lambda x: x.split(',')[-1].strip())

#  Extract only Celsius from values like '−7.3 (18.9)' and handle Unicode minus
def extract_celsius(temp_str):
    if not isinstance(temp_str, str):
        return None
    # Replace Unicode minus with standard minus
    clean_str = temp_str.replace('−', '-').strip()
    match = re.match(r"(-?\d+\.?\d*)", clean_str)
    return float(match.group(1)) if match else None

# Apply to high and low temp columns
weather_df['annual_high_c'] = weather_df['Annual(Avg. high °C (°F))'].apply(extract_celsius)
weather_df['annual_low_c'] = weather_df['Annual(Avg. low °C (°F))'].apply(extract_celsius)

#  Calculate average annual temp
weather_df['avg_annual_temp'] = weather_df[['annual_high_c', 'annual_low_c']].mean(axis=1)

#  Get average by province
weather_summary = weather_df.groupby('province')['avg_annual_temp'].mean().reset_index()
weather_summary.columns = ['province', 'avg_temperature_c']

print(" Clean Weather Summary by Province:")
display(weather_summary)


# **Merge Cheese & Weather Datasets**

In [ ]:
# At first lets check column names in both df
print("Cheese DataFrame columns:")
print(cheese_df.columns.tolist())

print("\n Weather Summary DataFrame columns:")
print(weather_summary.columns.tolist())


In [ ]:
# Force-rename cheese columns to lowercase for merging and analysis
cheese_df.rename(columns={
    'ManufacturerProvCode': 'province',
    'MoisturePercent': 'moisture_percent',
    'FatLevel': 'fat_level',
    'CategoryTypeEn': 'category_type',
    'MilkTypeEn': 'milk_type',
    'FlavourEn': 'flavour',
    'CheeseName': 'cheese_name',
}, inplace=True)

# Standardize province column casing in both DataFrames
cheese_df['province'] = cheese_df['province'].str.strip().str.title()
weather_summary['province'] = weather_summary['province'].str.strip().str.title()


In [ ]:
# Merge cheese and weather data on province code
merged_df = pd.merge(cheese_df, weather_summary, on='province', how='inner')

print(f" Merged dataset shape: {merged_df.shape}")
display(merged_df.head())


# **Verify Merge Quality**

In [ ]:
print("Unique provinces in merged data:")
print(merged_df['province'].unique())

print("\n Preview of average temperature distribution:")
print(merged_df['avg_temperature_c'].describe())


# **Visualizations + Analytical Insight**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# === 1. Cheese Category vs Avg Temp (Box Plot) ===
plt.figure(figsize=(12, 6))
sns.boxplot(data=merged_df, x='category_type', y='avg_temperature_c', palette='viridis')
plt.title('Cheese Category vs. Average Provincial Temperature', fontsize=14)
plt.xlabel('Cheese Category')
plt.ylabel('Avg Temperature (°C)')
plt.xticks(rotation=30)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# === 2. Fat & Moisture vs Temperature (Scatter Plot) ===
plt.figure(figsize=(14, 6))

# Fat content
plt.subplot(1, 2, 1)
sns.scatterplot(data=merged_df, x='avg_temperature_c', y='fat_level', alpha=0.7)
plt.title('Fat Level vs. Average Temperature')
plt.xlabel('Avg Temperature (°C)')
plt.ylabel('Fat Level')

# Moisture
plt.subplot(1, 2, 2)
sns.scatterplot(data=merged_df, x='avg_temperature_c', y='moisture_percent', color='teal', alpha=0.7)
plt.title('Moisture % vs. Average Temperature')
plt.xlabel('Avg Temperature (°C)')
plt.ylabel('Moisture (%)')

plt.tight_layout()
plt.show()


##  Final Insights & Reflection

###  Cheese Category vs Temperature
- **Warmer climates** (avg 6–7°C) favor **Hard** and **Veined cheeses**, which typically require more aging — potentially due to favorable humidity and temperature control.
- **Cooler regions** produce more **Fresh, Soft, and Semi-soft cheeses**, likely due to simpler processing and preservation needs in those climates.
- This aligns with production logic: softer cheeses require less maturation and may better fit shorter warm seasons or refrigeration-heavy environments.

###  Fat and Moisture vs Climate
- **Fat content** did not show any noticeable correlation with temperature. However, a more detailed breakdown by fat percentage may uncover hidden trends.
- **Moisture levels** showed a wider distribution in warmer regions — possibly due to diverse production styles or aging methods.
- There is **no strong linear correlation**, but **temperature may indirectly influence texture and shelf life preferences**.

###  Takeaway
This analysis highlights subtle yet meaningful links between **climate and cheese characteristics** in Canada. With more granular data (e.g., exact production volume, humidity), predictive models could help optimize cheese production by region.



# **Interactive Visualization (Plotly)**

In [ ]:
import plotly.express as px

fig = px.box(
    merged_df,
    x="category_type",
    y="avg_temperature_c",
    color="category_type",
    title="Interactive: Cheese Category vs Avg Provincial Temperature",
    labels={"avg_temperature_c": "Avg Temp (°C)", "category_type": "Cheese Category"},
    template="plotly_white"
)
fig.update_layout(showlegend=False)
fig.show()


# **Correlation Heatmap**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Encode fat level numerically (optional: for richer analysis)
if 'fat_level' in merged_df.columns:
    merged_df['fat_level_encoded'] = merged_df['fat_level'].map({'lower fat': 0, 'higher fat': 1})

# Select only numeric columns for correlation
numeric_cols = merged_df.select_dtypes(include=['float64', 'int64']).copy()
correlation_matrix = numeric_cols.corr()

# Plot heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title("Correlation Heatmap: Numeric Cheese Features")
plt.show()


# **Additional Visualizations**

In [ ]:
#Bar Plot: Cheese Category Frequency by Province
plt.figure(figsize=(14, 6))
sns.countplot(data=merged_df, x='province', hue='category_type', palette='Spectral')
plt.title('Cheese Category Distribution by Province')
plt.xlabel('Province')
plt.ylabel('Cheese Count')
plt.xticks(rotation=45)
plt.legend(title='Cheese Category')
plt.tight_layout()
plt.show()


###  Cheese Category Distribution by Province

This grouped bar chart clearly shows that **Quebec dominates cheese production** in Canada, with high counts in nearly every category, especially **Firm, Semi-soft, and Soft cheeses**.

Other provinces like **Ontario** and **British Columbia** contribute modestly with diverse categories, while provinces like **PEI and Alberta** show minimal representation.

> **Takeaway**: Cheese production in Canada is geographically concentrated, with Quebec acting as the core hub. This likely reflects historical, cultural, and infrastructural factors that favor artisanal and industrial-scale cheese-making in the province.


In [ ]:
#Temperature vs Number of Cheese Entries (Line Chart)
# Count number of cheese entries per province
cheese_counts = merged_df.groupby('province').size().reset_index(name='cheese_count')
cheese_temp_counts = pd.merge(cheese_counts, weather_summary, on='province')

# Plot
plt.figure(figsize=(10, 5))
sns.lineplot(data=cheese_temp_counts, x='avg_temperature_c', y='cheese_count', marker='o', color='darkblue')
plt.title('Number of Cheese Types vs. Avg Provincial Temperature')
plt.xlabel('Avg Temperature (°C)')
plt.ylabel('Number of Cheese Entries')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


###  Number of Cheese Types vs. Avg Provincial Temperature

This line plot highlights a surprising trend — the **largest number of cheese varieties are produced in regions with an average temperature around 4°C**.

Temperatures higher or lower than that tend to be associated with significantly **fewer cheese types**, suggesting an **optimal climatic range for cheese diversity**.

> **Interpretation**: There may be a "sweet spot" for cheese innovation and production diversity around moderate climates, which balance environmental control with natural maturation conditions.


In [ ]:
#  2D Scatter: Moisture vs Fat with Temp Color
import matplotlib.cm as cm

plt.figure(figsize=(10, 6))
sc = plt.scatter(
    merged_df['moisture_percent'],
    merged_df['fat_level'].map({'lower fat': 0, 'higher fat': 1}),
    c=merged_df['avg_temperature_c'],
    cmap='coolwarm',
    s=60,
    edgecolor='k',
    alpha=0.8
)
plt.colorbar(sc, label='Avg Temperature (°C)')
plt.title('Moisture vs. Fat Level (Colored by Temperature)')
plt.xlabel('Moisture (%)')
plt.ylabel('Fat Level (0 = lower fat, 1 = higher fat)')
plt.grid(True)
plt.tight_layout()
plt.show()


###  Moisture vs. Fat Level (Colored by Temperature)

This multivariate scatter plot reveals:

- **Higher moisture cheeses** are found across all temperatures but are more common in **warmer regions** (red-orange dots).
- **Lower fat cheeses** appear across all moisture levels and temperatures, while **higher fat cheeses** tend to cluster slightly in mid-moisture zones.
- There's **no obvious linear trend**, but the color gradient shows that warmer regions offer **greater variability** in cheese styles.

> **Insight**: While fat level may be more recipe-driven, **moisture interacts subtly with climate**, affecting cheese texture, storage needs, and shelf stability.


# **For Bonus Points**

In [ ]:
def clean_cheese_data(df, rename_map=None, drop_missing_thresh=0.3, required_fields=None):
    """
    Cleans the Canadian Cheese dataset dynamically:
    - Renames columns if rename_map is provided
    - Drops columns with > drop_missing_thresh missing values
    - Drops rows with nulls in required_fields (if provided)
    - Normalizes 'fat_level' and 'province' if present
    """
    df = df.copy()

    # Rename columns
    if rename_map:
        valid_renames = {k: v for k, v in rename_map.items() if k in df.columns}
        df.rename(columns=valid_renames, inplace=True)

    #Drop high-missing columns
    missing_ratio = df.isnull().mean()
    high_missing_cols = missing_ratio[missing_ratio > drop_missing_thresh].index.tolist()
    df.drop(columns=high_missing_cols, inplace=True, errors='ignore')

    # Drop rows missing required fields
    if required_fields:
        existing = [col for col in required_fields if col in df.columns]
        df.dropna(subset=existing, inplace=True)

    # Normalize text fields
    if 'fat_level' in df.columns:
        df['fat_level'] = df['fat_level'].astype(str).str.strip().str.lower()
    if 'province' in df.columns:
        df['province'] = df['province'].astype(str).str.strip().str.title()

    return df


In [ ]:
# define renaming and required fields
cheese_rename_map = {
    'ManufacturerProvCode': 'province',
    'MoisturePercent': 'moisture_percent',
    'FatLevel': 'fat_level',
    'CategoryTypeEn': 'category_type',
    'MilkTypeEn': 'milk_type',
    'FlavourEn': 'flavour',
    'CheeseName': 'cheese_name',
    'CharacteristicsEn': 'characteristics',
    'MilkTreatmentTypeEn': 'milk_treatment',
    'ManufacturingTypeEn': 'manufacturing_type'
}

required_fields = ['moisture_percent', 'category_type', 'milk_type']

# Apply pipeline
cheese_df_cleaned = clean_cheese_data(
    df=cheese_df,
    rename_map=cheese_rename_map,
    drop_missing_thresh=0.3,
    required_fields=required_fields
)


##  Bonus: Custom Data Cleaning Pipeline

To demonstrate advanced skills in real-world data preparation, I implemented a **modular and reusable data cleaning pipeline** using a Python function.

### Features of the Cleaning Function:
- **Dynamic column renaming** (based on actual schema)
- **Auto-detection of high-missing columns** with a threshold
- **Safe null row removal** for required fields
- **Text normalization** for consistency in categorical columns

### Why This Matters:
- Real data projects often deal with **inconsistent column names, schema drift, and missing data**.
- This pipeline anticipates those issues and adapts accordingly.
- It allows future cheese datasets (or any similar structured data) to be cleaned with **one line of reusable code**.

> This isn't just functional — it's designed with **maintainability, readability, and reusability** in mind, which are hallmarks of professional data engineering.
